In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
from keras.utils import np_utils

In [2]:
IMG_SIZE = 100
x = [] #features
y = [] #labels
source = 'left_eyes_data'
for filename in os.listdir(source):
    string = filename.split(".")
    img = cv2.imread(os.path.join(source,filename))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x.append(img)
    y.append([int(string[0]), int(string[1])])
print(x)
print(y)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
x = np.array(x).reshape(len(x), 100*100*1)
x = x.astype('float32')
x /= 255

In [4]:
y = np.array(y)

In [5]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [6]:
x_train.shape

(10062, 10000)

In [7]:
y_train.shape

(10062, 2)

In [8]:
x_test.shape

(2516, 10000)

In [9]:
y_test.shape

(2516, 2)

In [10]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense

In [11]:
model = Sequential()
model.add(Dense(32, input_dim=10000, kernel_initializer='he_uniform', activation='relu'))
model.add(Dense(2, kernel_initializer='he_uniform'))
model.compile(loss='mae', optimizer='adam', metrics=['accuracy'])

In [12]:
model.fit(x, y, verbose=0, epochs=100)

In [13]:
model.evaluate(x = x, y = y)

394/394 [==============================] - 3s 5ms/step - loss: 85.4450 - accuracy: 0.9105


[85.44499969482422, 0.9104785919189453]

In [14]:
predictions = model.predict(x_test)

In [15]:
predictions

array([[ 746.0702  ,  256.80325 ],
       [ 552.5902  ,   68.865845],
       [ 580.0335  ,  345.3765  ],
       ...,
       [1041.078   ,  502.59036 ],
       [ 636.6777  ,  504.01495 ],
       [1036.4557  ,  105.34958 ]], dtype=float32)

In [16]:
y_test

array([[ 720,  300],
       [ 540,   40],
       [ 600,  460],
       ...,
       [1240,  500],
       [ 700,  740],
       [ 998,    0]])

In [17]:
model_json = model.to_json()
with open("new_model.json", "w") as json_file:
    json_file.write(model_json)

model.save_weights("new_model.h5")
print("Saved model to disk")

Saved model to disk


In [1]:
from keras.models import model_from_json
from tkinter import * 
import threading
import pandas as pd
import numpy as np
import cv2
import os

json_file = open('new_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("new_model.h5")
print("Loaded model from disk")

Loaded model from disk


In [3]:
x_predict = 0
y_predict = 0

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_eye.xml')

cap = cv2.VideoCapture(0)
_, img = cap.read()

def show_frame():
    global img, x_predict, y_predict
    while True:
        _, img = cap.read()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(img, 1.3, 5)
        for (x,y,w,h) in faces:
            roi_gray = img[y:y+h, x:x+w]
            eyes = eye_cascade.detectMultiScale(roi_gray, 1.3, 5) # detect eyes
            width = np.size(roi_gray, 1) # get face frame width
            height = np.size(roi_gray, 0) # get face frame height
            for (x, y, w, h) in eyes:
                if y > height / 2:
                    pass
                eyecenter = x + w / 2  # get the eye center
                if eyecenter < width * 0.5:
                    left_eye = roi_gray[y:y + h, x:x + w]
                    left_eye = cv2.resize(left_eye, dsize=(100, 100))
                    cv2.imshow("eye", left_eye)
                    left_eye = np.array(left_eye).reshape(-1, 10000)
                    left_eye = left_eye.astype('float32')
                    left_eye /= 255
                    prediction = loaded_model.predict([left_eye])
                    x_predict = prediction[0][0]
                    y_predict = prediction[0][1]
        interrupt = cv2.waitKey(10)
        if interrupt & 0xFF == 27: # esc key
            break

def tkinter_frame():
    global myCanvas
    frame = Tk()  
    frame.attributes('-fullscreen',True) 
    
    myCanvas = Canvas(frame)
    myCanvas.pack(fill=BOTH, expand=True)
    
    frame.mainloop()

def pointer():
    global x_predict, y_predict
    while True:
        create_circle(x_predict, y_predict, 20)
        
def create_circle(x, y, r): #center coordinates, radius
    global myCanvas
    x0 = x - r
    y0 = y - r
    x1 = x + r
    y1 = y + r
    try:
        dot = myCanvas.create_oval(x0, y0, x1, y1, fill="red")  
    except:
        pass
        

t1 = threading.Thread(target=show_frame)
t2 = threading.Thread(target=tkinter_frame)
t3 = threading.Thread(target=pointer)

# starting thread 1
t1.start()
# starting thread 2
t2.start()
t3.start()

# wait until thread 1 is completely executed
# t1.join()
# wait until thread 2 is completely executed
t2.join() 